In [ ]:
import numpy as np
from ansys.mapdl.core import launch_mapdl

# === 启动 MAPDL ===
mapdl = launch_mapdl(
    nproc=4,
    jobname="bfv_case",
    run_location="E:/Python/PyMAPDL",
    loglevel="INFO",
    override=True
)
print(mapdl.version)
mapdl.clear()
mapdl.prep7()
mapdl.units("SI")

# === 材料定义 ===
mapdl.mp("DENS", 1, 2400)
mapdl.mptemp(1, 100)
mapdl.mptemp(2, 150)
mapdl.mptemp(3, 200)
mapdl.mptemp(4, 300)
mapdl.mptemp(5, 400)

#Specific heat
mapdl.mpdata("C", 1, 1, 900)
mapdl.mpdata("C", 1, 2, 950)
mapdl.mpdata("C", 1, 3, 1000)
mapdl.mpdata("C", 1, 4, 1050)
mapdl.mpdata("C", 1, 5, 1100)

#Thermal conductivity
mapdl.mpdata("KXX", 1, 1, 2.5)
mapdl.mpdata("KXX", 1, 2, 2.463)
mapdl.mpdata("KXX", 1, 3, 2.396)
mapdl.mpdata("KXX", 1, 4, 2.259)
mapdl.mpdata("KXX", 1, 5, 2.0)

# === 几何参数 ===
od_media = 0.254
od_hx = 0.0508
t_hx = 0.00635
id_hx = od_hx - 2 * t_hx
length = 0.508
num_radial = 10
num_circ = 9

# === 关键点和线 ===
inner_kps = []
outer_kps = []
for i in range(num_circ):
    angle = i * (90 / num_circ)
    x_inner = (id_hx / 2) * np.cos(np.deg2rad(angle))
    y_inner = (id_hx / 2) * np.sin(np.deg2rad(angle))
    x_outer = (od_media / 2) * np.cos(np.deg2rad(angle))
    y_outer = (od_media / 2) * np.sin(np.deg2rad(angle))
    inner_kps.append(mapdl.k("", x_inner, y_inner, 0))
    outer_kps.append(mapdl.k("", x_outer, y_outer, 0))

inner_arcs = [mapdl.l(inner_kps[i], inner_kps[(i+1)%num_circ]) for i in range(num_circ)]
outer_arcs = [mapdl.l(outer_kps[i], outer_kps[(i+1)%num_circ]) for i in range(num_circ)]
radial_lines = [mapdl.l(inner_kps[i], outer_kps[i]) for i in range(num_circ)]
areas = []
for i in range(num_circ):
    next_i = (i+1) % num_circ
    lines = [inner_arcs[i], radial_lines[next_i], outer_arcs[i], radial_lines[i]]
    areas.append(mapdl.al(*lines))

# === 单元类型和网格设置 ===
mapdl.et(1, "SOLID70")
volumes = [mapdl.vext(a, dz=length) for a in areas]

mapdl.lsel("S", "LINE", "", min(radial_lines), max(radial_lines))
mapdl.lesize("ALL", "", "", num_radial, 1)

num_z = 10
mapdl.lsel("S", "LOC", "X", 0)
mapdl.lsel("R", "LOC", "Y", 0)
mapdl.lsel("R", "LOC", "Z", 0, length)
mapdl.lesize("ALL", "", "", num_z, 1)
mapdl.allsel()
mapdl.vmesh("ALL")

# === 选择靠内圈节点 ===
inner_radius = id_hx / 2
points = mapdl.mesh.grid.points
radii = np.sqrt(points[:, 0]**2 + points[:, 1]**2)
final_node_indices = np.where(np.abs(radii - inner_radius) < 1e-3)[0]
final_nodes = final_node_indices + 1

# === 定义温度表格并用 BFV 一次性加载 ===
time_steps = np.linspace(0, 14400, 25)
temp_load = np.linspace(100, 400, 25)

# 创建 MAPDL table（BFV 用法）

mapdl.input_strings([
    "*DIM, TEMP_TABLE, TABLE, 25, 1, , TIME",
    *[f"TEMP_TABLE({i+1},1) = {T}" for i, T in enumerate(temp_load)]
])


# === 求解设置 ===
mapdl.slashsolu()

# 应用温度表
mapdl.nsel("NONE")
for node in final_nodes:
    mapdl.nsel("A", "NODE", "", node)
mapdl.bfv("ALL", "TEMP", "TEMP_TABLE")
mapdl.nsel("ALL")

mapdl.antype("TRANS")         # Transient analysis (等同于 4)
mapdl.trnopt("FULL")          # Full transient
mapdl.kbc(0)                  # Ramped loading
mapdl.timint("ON")            # Enable time integration
mapdl.ic("ALL", "TEMP", 100)  # Initial temperature = 100°C

mapdl.autots("OFF")           # 手动控制时间步
mapdl.deltim(600)             # 每步600秒，24步刚好14400秒
mapdl.nsubst(24)              # 24 个子步
mapdl.time(14400)             # 总时间

# ✅ 明确输出设置：保存所有时间步的温度结果
mapdl.outres("ALL", "ALL")   # 强制每步保存温度结果
mapdl.allsel()                # 确保选择全部实体

# ✅ 重要：确保输入 BFV 后才执行 solve
mapdl.solve()

mapdl.finish()


INFO - GRPC_127.0.0.1:50053 -  mapdl_core - run - PARAMETER  = E:\Python\PyMAPDL
INFO - GRPC_127.0.0.1:50053 -  mapdl_core - run - /SHOW SWITCH PLOTS TO  PNG         - RASTER MODE.
INFO - GRPC_127.0.0.1:50053 -  mapdl_core - run - PARAMETER  = E:\Python\PyMAPDL
INFO - GRPC_127.0.0.1:50053 -  mapdl_core - run - PARAMETER  = E:\Python\PyMAPDL
INFO - GRPC_127.0.0.1:50053 -  mapdl_grpc - __init__ - Connected to MAPDL server running at localhost on 127.0.0.1:50053 on windows OS


25.1


INFO - GRPC_127.0.0.1:50053 -  mapdl_core - run - *****MAPDL VERIFICATION RUN ONLY*****
     DO NOT USE RESULTS FOR PRODUCTION

          ***** MAPDL ANALYSIS DEFINITION (PREP7) *****
INFO - GRPC_127.0.0.1:50053 -  mapdl_core - run - SI UNITS SPECIFIED FOR INTERNAL    
  LENGTH        (l)  = METER (M)
  MASS          (M)  = KILOGRAM (KG)
  TIME          (t)  = SECOND (SEC)
  TEMPERATURE   (T)  = KELVIN (K)
  TOFFSET            = 0.0 KELVIN
  CHARGE        (Q)  = COULOMB
  FORCE         (f)  = NEWTON (N) (KG-M/SEC2)
  HEAT               = JOULE (N-M)

  PRESSURE           = PASCAL (NEWTON/M**2)
  ENERGY        (W)  = JOULE (N-M)
  POWER         (P)  = WATT (N-M/SEC)
  CURRENT       (i)  = AMPERE (COULOMBS/SEC)
  CAPACITANCE   (C)  = FARAD
  INDUCTANCE    (L)  = HENRY
  MAGNETIC FLUX      = WEBER
  RESISTANCE    (R)  = OHM
  ELECTRIC POTENTIAL = VOLT

INFO - GRPC_127.0.0.1:50053 -  mapdl_core - run - MATERIAL          1     DENS =   2400.000
INFO - GRPC_127.0.0.1:50053 -  mapdl_core - ru

FINISH SOLUTION PROCESSING


 ***** ROUTINE COMPLETED *****  CP =         0.000

In [6]:
mapdl.post1()

grid = mapdl.mesh.grid  # PyVista-compatible mesh
nodes = mapdl.mesh.nodes
coords = grid.points  # Nx3 array of (x, y, z)


for nid, coord in zip(nodes, coords):
    print(f"Node {nid}: x={coord[0]:.4f}, y={coord[1]:.4f}, z={coord[2]:.4f}")

# 尝试通过 XYZ 直接找最近节点
inner_radius = id_hx / 2
outer_radius = od_media / 2

# Select nearest to (inner_radius, 0, z_center)
mapdl.nsel("S", "LOC", "X", inner_radius - 1e-3, inner_radius + 1e-3)
mapdl.nsel("R", "LOC", "Y", -1e-3, 1e-3)
mapdl.nsel("R", "LOC", "Z", -1e-3, 1e-3)
inner_node = int(mapdl.get("INNER", "NODE", 0, "NUM", "MAX"))

# Select nearest to (outer_radius, 0, z_center)
mapdl.nsel("S", "LOC", "X", outer_radius - 1e-3, outer_radius + 1e-3)
mapdl.nsel("R", "LOC", "Y", -1e-3, 1e-3)
mapdl.nsel("R", "LOC", "Z", -1e-3, 1e-3)
outer_node = int(mapdl.get("OUTER", "NODE", 0, "NUM", "MAX"))

mapdl.allsel()
mapdl.finish()

print(f"inner_node = {inner_node}, outer_node = {outer_node}")


INFO - GRPC_127.0.0.1:50053 -  mapdl_core - run - *****MAPDL VERIFICATION RUN ONLY*****
     DO NOT USE RESULTS FOR PRODUCTION

          ***** MAPDL RESULTS INTERPRETATION (POST1) *****

 *** NOTE ***                            CP =       0.000   TIME= 00:00:00
 Reading results into the database (SET command) will update the current 
 displacement and force boundary conditions in the database with the     
 values from the results file for that load set.  Note that any          
 subsequent solutions will use these values unless action is taken to    
 either SAVE the current values or not overwrite them (/EXIT,NOSAVE).
INFO - GRPC_127.0.0.1:50053 -  mapdl_core - run - LIST ALL SELECTED COMPONENTS
INFO - GRPC_127.0.0.1:50053 -  mapdl_core - run - *GET  __FLOATPARAMETER__  FROM  ELEM  ITEM=COUN           VALUE=  4500.00000
INFO - GRPC_127.0.0.1:50053 -  mapdl_core - run - SELECT ALL ENTITIES OF TYPE= ALL  AND BELOW
INFO - GRPC_127.0.0.1:50053 -  mapdl_core - run - UNSELECT ALL COMPONEN

Node [0.01905 0.      0.     ]: x=0.0190, y=0.0000, z=0.0000
Node [0.01876059 0.003308   0.        ]: x=0.0188, y=0.0033, z=0.0000
Node [0.01895353 0.00110267 0.        ]: x=0.0190, y=0.0011, z=0.0000
Node [0.01885706 0.00220533 0.        ]: x=0.0189, y=0.0022, z=0.0000
Node [0.12507058 0.02205332 0.        ]: x=0.1251, y=0.0221, z=0.0000
Node [0.02939159 0.00518253 0.        ]: x=0.0294, y=0.0052, z=0.0000
Node [0.04002259 0.00705706 0.        ]: x=0.0400, y=0.0071, z=0.0000
Node [0.05065359 0.00893159 0.        ]: x=0.0507, y=0.0089, z=0.0000
Node [0.06128459 0.01080613 0.        ]: x=0.0613, y=0.0108, z=0.0000
Node [0.07191559 0.01268066 0.        ]: x=0.0719, y=0.0127, z=0.0000
Node [0.08254659 0.01455519 0.        ]: x=0.0825, y=0.0146, z=0.0000
Node [0.09317759 0.01642972 0.        ]: x=0.0932, y=0.0164, z=0.0000
Node [0.10380859 0.01830425 0.        ]: x=0.1038, y=0.0183, z=0.0000
Node [0.11443958 0.02017879 0.        ]: x=0.1144, y=0.0202, z=0.0000
Node [0.127 0.    0.   ]: x=0

INFO - GRPC_127.0.0.1:50053 -  mapdl_core - run - SELECT       FOR ITEM=LOC  COMPONENT=X     BETWEEN 0.18050E-01 AND  0.20050E-01
   KABS=  0.  TOLERANCE= 0.200000E-10

        252  NODES (OF       6765  DEFINED) SELECTED BY  NSEL  COMMAND.


Node [0.03308209 0.07670677 0.13546667]: x=0.0331, y=0.0767, z=0.1355
Node [0.03308209 0.07670677 0.16933333]: x=0.0331, y=0.0767, z=0.1693
Node [0.03308209 0.07670677 0.2032    ]: x=0.0331, y=0.0767, z=0.2032
Node [0.03308209 0.07670677 0.23706667]: x=0.0331, y=0.0767, z=0.2371
Node [0.03308209 0.07670677 0.27093333]: x=0.0331, y=0.0767, z=0.2709
Node [0.03308209 0.07670677 0.3048    ]: x=0.0331, y=0.0767, z=0.3048
Node [0.03308209 0.07670677 0.33866667]: x=0.0331, y=0.0767, z=0.3387
Node [0.03308209 0.07670677 0.37253333]: x=0.0331, y=0.0767, z=0.3725
Node [0.03308209 0.07670677 0.4064    ]: x=0.0331, y=0.0767, z=0.4064
Node [0.03308209 0.07670677 0.44026667]: x=0.0331, y=0.0767, z=0.4403
Node [0.03308209 0.07670677 0.47413333]: x=0.0331, y=0.0767, z=0.4741
Node [0.04232508 0.08426233 0.03386667]: x=0.0423, y=0.0843, z=0.0339
Node [0.04232508 0.08426233 0.06773333]: x=0.0423, y=0.0843, z=0.0677
Node [0.04232508 0.08426233 0.1016    ]: x=0.0423, y=0.0843, z=0.1016
Node [0.04232508 0.0

INFO - GRPC_127.0.0.1:50053 -  mapdl_core - run - RESELECT     FOR ITEM=LOC  COMPONENT=Y     BETWEEN-0.10000E-02 AND  0.10000E-02
   KABS=  0.  TOLERANCE= 0.200000E-10

         31  NODES (OF       6765  DEFINED) SELECTED BY  NSEL  COMMAND.
INFO - GRPC_127.0.0.1:50053 -  mapdl_core - run - RESELECT     FOR ITEM=LOC  COMPONENT=Z     BETWEEN-0.10000E-02 AND  0.10000E-02
   KABS=  0.  TOLERANCE= 0.200000E-10

          1  NODES (OF       6765  DEFINED) SELECTED BY  NSEL  COMMAND.
INFO - GRPC_127.0.0.1:50053 -  mapdl_core - run - *GET  INNER     FROM  NODE  ITEM=NUM  MAX   VALUE=  1.00000000
INFO - GRPC_127.0.0.1:50053 -  mapdl_core - run - SELECT       FOR ITEM=LOC  COMPONENT=X     BETWEEN 0.12600     AND  0.12800    
   KABS=  0.  TOLERANCE= 0.200000E-10

         47  NODES (OF       6765  DEFINED) SELECTED BY  NSEL  COMMAND.
INFO - GRPC_127.0.0.1:50053 -  mapdl_core - run - RESELECT     FOR ITEM=LOC  COMPONENT=Y     BETWEEN-0.10000E-02 AND  0.10000E-02
   KABS=  0.  TOLERANCE= 0.200000E

inner_node = 1, outer_node = 15


In [7]:
mapdl.post1()

max_set = 0
while True:
    try:
        mapdl.set(max_set + 1)
        max_set += 1
    except Exception as e:
        break

print(f"🟢 共找到 {max_set} 个时间步")


INFO - GRPC_127.0.0.1:50053 -  mapdl_core - run - *****MAPDL VERIFICATION RUN ONLY*****
     DO NOT USE RESULTS FOR PRODUCTION

          ***** MAPDL RESULTS INTERPRETATION (POST1) *****
INFO - GRPC_127.0.0.1:50053 -  mapdl_core - run - USE LOAD STEP     1  SUBSTEP  LAST    FOR LOAD CASE 0 

 SET COMMAND GOT LOAD STEP=     1  SUBSTEP=    24  CUMULATIVE ITERATION=    24
   TIME/FREQUENCY=  14400.    
 TITLE=


🟢 共找到 1 个时间步


In [8]:
import pandas as pd

# 进入 POST1 模块
mapdl.post1()

# 获取总的时间步数
mapdl.set(1)  # 先设置一个步，确保 SET 可用
nsets = int(mapdl.get("NSETS", "ACTIVE", 0, "SET", "NUM"))
print(f"共保存 {nsets} 个时间步")

# 初始化数据存储
time_data = []
inner_temps = []
outer_temps = []

# 遍历每个时间步
for i in range(1, nsets + 1):
    mapdl.set(i)  # 设置到第 i 个时间步

    time_val = mapdl.get("_T", "TIME", 0)  # 当前时间
    temp_inner = mapdl.get("_TIN", "NODE", inner_node, "TEMP")  # 内节点温度
    temp_outer = mapdl.get("_TOU", "NODE", outer_node, "TEMP")  # 外节点温度

    time_data.append(time_val)
    inner_temps.append(temp_inner)
    outer_temps.append(temp_outer)

    print(f"Step {i}: Time={time_val:.1f}s, Inner Temp={temp_inner:.2f}°C, Outer Temp={temp_outer:.2f}°C")

# 保存为 DataFrame 并导出 CSV
df = pd.DataFrame({
    "Time (s)": time_data,
    "Inner Temp (C)": inner_temps,
    "Outer Temp (C)": outer_temps
})
df.to_csv("E:/Python/PyMAPDL/temperature_results.csv", index=False)
print("✅ 温度数据已保存到 temperature_results.csv")


MapdlExitedError: The MAPDL instance has been exited before running the command: /POST1,

In [ ]:
import numpy as np
from ansys.mapdl.core import launch_mapdl

# === 启动 MAPDL ===
mapdl = launch_mapdl(
    nproc=4,
    jobname="bfv_case",
    run_location="E:/Python/PyMAPDL",
    loglevel="ERROR",
    override=True
)

mapdl.clear()
mapdl.prep7()
mapdl.units("SI")

# === 材料定义 ===
mapdl.mp("DENS", 1, 2400)
mapdl.mptemp(1, 100)
mapdl.mptemp(2, 150)
mapdl.mptemp(3, 200)
mapdl.mptemp(4, 300)
mapdl.mptemp(5, 400)
mapdl.mpdata("C", 1, 1, 900)
mapdl.mpdata("C", 1, 2, 950)
mapdl.mpdata("C", 1, 3, 1000)
mapdl.mpdata("C", 1, 4, 1050)
mapdl.mpdata("C", 1, 5, 1100)
mapdl.mpdata("KXX", 1, 1, 2.5)
mapdl.mpdata("KXX", 1, 2, 2.463)
mapdl.mpdata("KXX", 1, 3, 2.396)
mapdl.mpdata("KXX", 1, 4, 2.259)
mapdl.mpdata("KXX", 1, 5, 2.0)

# === 几何参数 ===
od_media = 0.254
od_hx = 0.0508
t_hx = 0.00635
id_hx = od_hx - 2 * t_hx
length = 0.508
num_radial = 10
num_circ = 9

# === 关键点和线 ===
inner_kps = []
outer_kps = []
for i in range(num_circ):
    angle = i * (90 / num_circ)
    x_inner = (id_hx / 2) * np.cos(np.deg2rad(angle))
    y_inner = (id_hx / 2) * np.sin(np.deg2rad(angle))
    x_outer = (od_media / 2) * np.cos(np.deg2rad(angle))
    y_outer = (od_media / 2) * np.sin(np.deg2rad(angle))
    inner_kps.append(mapdl.k("", x_inner, y_inner, 0))
    outer_kps.append(mapdl.k("", x_outer, y_outer, 0))

inner_arcs = [mapdl.l(inner_kps[i], inner_kps[(i+1)%num_circ]) for i in range(num_circ)]
outer_arcs = [mapdl.l(outer_kps[i], outer_kps[(i+1)%num_circ]) for i in range(num_circ)]
radial_lines = [mapdl.l(inner_kps[i], outer_kps[i]) for i in range(num_circ)]
areas = []
for i in range(num_circ):
    next_i = (i+1) % num_circ
    lines = [inner_arcs[i], radial_lines[next_i], outer_arcs[i], radial_lines[i]]
    areas.append(mapdl.al(*lines))

# === 单元类型和网格设置 ===
mapdl.et(1, "SOLID70")
volumes = [mapdl.vext(a, dz=length) for a in areas]

mapdl.lsel("S", "LINE", "", min(radial_lines), max(radial_lines))
mapdl.lesize("ALL", "", "", num_radial, 1)

num_z = 10
mapdl.lsel("S", "LOC", "X", 0)
mapdl.lsel("R", "LOC", "Y", 0)
mapdl.lsel("R", "LOC", "Z", 0, length)
mapdl.lesize("ALL", "", "", num_z, 1)
mapdl.allsel()
mapdl.vmesh("ALL")

# === 选择靠内圈节点 ===
inner_radius = id_hx / 2
points = mapdl.mesh.grid.points
radii = np.sqrt(points[:, 0]**2 + points[:, 1]**2)
final_node_indices = np.where(np.abs(radii - inner_radius) < 1e-3)[0]
final_nodes = final_node_indices + 1

# === 定义温度表格 ===
time_steps = np.linspace(0, 14400, 25)
temp_load = np.linspace(100, 400, 25)

# 更可靠的表定义方式
mapdl.run("*DIM, TEMP_TABLE, TABLE, 25, 1, , TIME")
for i, (time_val, temp_val) in enumerate(zip(time_steps, temp_load), start=1):
    mapdl.run(f"TEMP_TABLE({i},0) = {time_val}")
    mapdl.run(f"TEMP_TABLE({i},1) = {temp_val}")

# === 求解设置 ===
mapdl.slashsolu()

# 应用温度表
mapdl.nsel("S", "NODE", "", *final_nodes)
mapdl.bfv("ALL", "TEMP", "%TEMP_TABLE%")  # 使用%包裹表名
mapdl.nsel("ALL")

mapdl.antype("TRANS")         # Transient analysis
mapdl.trnopt("FULL")          # Full transient
mapdl.kbc(0)                  # Ramped loading
mapdl.timint("ON")            # Enable time integration
mapdl.ic("ALL", "TEMP", 100)  # Initial temperature = 100°C

# 更清晰的时间步控制
mapdl.autots("ON")            # 启用自动时间步
mapdl.deltim(600, 300, 1200)  # 初始600s，最小300s，最大1200s
mapdl.time(14400)             # 总时间

# 输出控制
mapdl.outres("ALL", "ALL")    # 保存所有结果
mapdl.allsel()

# 解决
mapdl.solve()
mapdl.finish()

# === 后处理 ===
mapdl.post1()
mapdl.set("LAST")  # 确保读取最后一步

# 获取时间步信息
num_sets = 0
while True:
    try:
        mapdl.set(num_sets + 1)
        num_sets += 1
    except:
        break

print(f"🟢 共找到 {num_sets} 个时间步")